# Test SSO Login

The instance has all the permissions required, now run the login command for the `Jupyter-IR-ViewOnly` role and the loggin account.

In [ ]:
from jupyterirtools import sso

sso.login("Jupyter-IR-ViewOnly")

In [ ]:
#session = boto3.Session(profile_name='ManagedPermissionSet-383086473915')
import boto3
boto3.setup_default_session()
sts = boto3.client('sts')
sts.get_caller_identity()

In [ ]:
! aws sts get-caller-identity


In [ ]:
sso.logout()

In [ ]:
boto3.setup_default_session()
sts = boto3.client('sts')
try:
    sts.get_caller_identity()
    print("The logout FAILED")
except:
    print("The logout was successful")